# Model notebook
This notebook will be used to download data from BQL and fine-tune the data sets to match our requirements.

## Quick Intro to BQL

BQL is a new API based on normalized, curated, point-in-time data. Using BQL, you can perform custom calculations directly in the Bloomberg Cloud. Specify the data you want and the calculation you want to perform—including, but not limited to, arithmetic and statistical operations, filtering, grouping, scoring, and dates-based analysis—so you can synthesize large amounts of data and extract the exact information you need.

In a single BQL query, you construct a universe of tickers (such as a list of peers) and define the data you want (such as earnings per share). You can apply analysis and calculations on the data directly on the server side, such as calculating an average or multiplying two values. You can also add optional parameters that fit the data to your model, such as the time period (if you want historical data) and currency (if you want to convert it).

See more details on [BQLX](https://blinks.bloomberg.com/screens/BQLX).

__TLDR__: BQL is the Bloomberg data source for our BQuant apps!

In [ ]:
# Mandatory libraries
import pandas as pd
import numpy as np
from collections import OrderedDict
import bql

`bq` will be our BQL service object to load metadata and execute BQL queries

In [ ]:
bq = None

Let's create a `_init_bql()` method to initialize the BQL service. This is useful to set it once for all during the life time of our App, to avoid restarting a new BQL connection every time a request is made. 

Note that we will use the below to interact with `bq` object while we are in this notebook. This will no longer be useful when building the App itself.
```python 
global bq
```

In [ ]:
# construct a function to start BQL service
def _init_bql():
    global bq
    """Loads self._bq from class-level shared BQL instance if no instance is available yet.
    Class-level shared BQL instance would be initialized if needed.
    """
    if bq is None:
        bq = bql.Service()
        

Test the connection.

In [ ]:
# initialize BQL service
_init_bql()

## Query retrieval methods
Whether we want to retrieve equity or fixed income data, we need to create a `get_data` function. <br/>
For conveniency, we will set an option based on the asset class queried, and allow to retrieve Bloomberg data via the query mode.<br/>

To do this, we will build 3 methods:
* `_get_data(query)` that retrieves data model based on the query string that is passed to it. Output is a DataFrame to be used in the `DatModel` for further manipulation. 
* `_combine_dfs(response)` that will combine every item retrieved as singular dataframe from BQL. Output is the raw data from BQL.
* `_yield_raw_df(df, asset_class)` that will render the raw dataframe into a manipulated table to fit in the App visuals.

In [ ]:
def _get_data(query):
    """
    Retrieve data model based on the universe and the BQL items
    Inputs:
    - query (str): BQL query string to be requested.
    """
    try:
        # add mode=cached for queries involving asset class universe screening
        query = '{} with(mode=cached)'.format(query)
        
        # request the query string
        r = bq.execute(query)

        # store the whole dataset in data
        data = _combine_dfs(r)

    except Exception as e:
        print('Error while fetching data ({})'.format(e))
        data = pd.DataFrame()

    return data


def _combine_dfs(response):
    """
    Concatenate in a DataFrame all the response (per column item) from BQL
    Inputs:
    - response (DataFrame): dataframe that contains the Bloomberg data
    retrieved via BQL without some dropped items
    """
    data = []
    drop_items = ['REVISION_DATE','AS_OF_DATE','PERIOD_END_DATE','CURRENCY','Partial Errors']
    for r in response:
        df = r.df().drop(drop_items, axis='columns', errors='ignore')
        data.append(df)
    return pd.concat(data, axis=1).reset_index()
    
    
def _yield_raw_df(df, asset_class):
    """
    Retrieve data model based on the universe and the BQL items
    Inputs:
    - df (DataFrame): dataframe that contains the Bloomberg data
    retrieved via BQL.
    - asset_class (str): asset class string working as a switch 
    to handle the data manipulation after retrieval.
    """
    if asset_class == 'Fixed Income':
        # trim the index and build additional columns used for pivoting tables
        col_mapping = {'YEAR(ANNOUNCE_DATE())':'Year',
                       'MONTH(ANNOUNCE_DATE())':'Month',
                       '#amt':'Amount Out',
                       'CNTRY_OF_RISK()':'Country'}
        df.rename(columns=col_mapping, inplace=True)

        # create the Year, Month and Announce Date columns:
        df['Year'] = df['Year'].astype('str').apply(lambda x: x.split('.')[0])
        df['Month'] = df['Month'].astype('str').apply(lambda x: '{:02d}'.format(int(x.split('.')[0])))
        df['Announce Date'] = df.apply(lambda x: '{}-{:02d}'.format(x['Year'],int(x['Month'])), axis=1)

        
    elif asset_class == 'Equity':
        # rename the columns to a more readable content
        col_mapping = {'COUNTRY_FULL_NAME()':'Country',
                       'GICS_SECTOR_NAME()':'Sector',
                       '#avg_rel_ret':'1m return'}
        df.rename(columns=col_mapping, inplace=True)
        
    else:
        print('Not Implemented')
        
    # mask NullGroup
    maskNullGroup = df.apply(lambda x: 'NullGroup' not in x.ID.split(':'), axis=1)
    df = df[maskNullGroup]
    df = df[df['Country'] != 'NA']
    
    return df

-------
## Query definition
We are using raw query to simplify the data acquisition implementation part. <br>
<i>Note that this can be transformed to be using the BQL data model. </i>

Based on the asset class you are interested in, you can load the 
* `bond_issuance` variable that will download the debt issued in USD since 01 Jan 2010 for all the bonds in the Bloomberg Barclays Global-Aggregate Total Return Index, or;
* `equity_score` variable will build a composite score for every stock and display them by sector and country for the MSCI ex Asia.

In [ ]:
bond_issuance = '''
    let(#amt=sum(group(amt_outstanding(currency='USD'),[year(announce_date()), month(announce_date()),cntry_of_risk()]))/1000000;) 
    get(#amt) 
    for( filter(members('LEGATRUU index'), announce_date() >= '2010-01-01') ) 
'''

equity_score = '''
    let(#ret1m = (product(dropna(1.0+day_to_day_total_return(start=-1m,end=0d)))-1)*100;
        #ret1m_idx = value(#ret1m,['BWORLD Index']);
        #rel_ret1m = #ret1m - #ret1m_idx;
        #avg_rel_ret = avg(group(#rel_ret1m,[country_full_name(),gics_sector_name()]));)
    get(#avg_rel_ret)
    for( members('BWORLD Index'))
'''

### 1. Fixed income query retrieval first
Let's fetch fixed income data first.

In [ ]:
# get data from `bond_issuance` query
raw_data = _get_data(bond_issuance)
# render the data in a readable format for the visuals
data = _yield_raw_df(raw_data, 'Fixed Income')
# display the bottom entries of the dataframe
data.tail()

Now we will create a function to pivot the data in such a way we will be providing our heatmap object with.<br/>
To that end, we will create the method `build_2dim_dataset` and we will reference the `x`-axis, `y`-axis, and `v` values to pivot the table accordingly. We will also mention the type of calculation for each cell, as bond issuance calculation will provide the sum of all the amount outstanding, and the equity returns by sector will be given as a median. 

In [ ]:
def build_2dim_dataset(df, x='Month', y='Year', v='Amount Out', calc_type='sum'):
    '''
    Summary: returns a table in 2-dim with data transformed and 
    aggregated by x and by y.
    Inputs:
        - df (DataFrame): dataframe that contains the BQL data cleaned 
        - x (str): columns on which to pivot the table to represent the x-axis
        - y (str): columns on which to pivot the table to represent the y-axis
        - calc_type (str): sum or median can be referred to aggregate the data
    '''

    # create the final dataframe
    output = df.pivot_table(index=y, columns=x, values=v, aggfunc=calc_type)
    
    return output

Let's have a look again at the names in the columns of the dataframe to setup the matrix data

In [ ]:
data.columns

We'll use then `Year` amd `Country` as the `x` and `y` axis. The values will be the amount outstanding. <br>
Note that we can refer to the `calc_type` based on type of aggregated values we want to display in the matrix.

In [ ]:
build_2dim_dataset(data, x='Month', y='Year', v='Amount Out', calc_type='sum')

### 2.Equity query retrieval
Let's now fetch equity data.

In [ ]:
# get data from `equity_score` query, render the data and display the bottom of the dataframe
raw_data2 = _get_data(equity_score)
data2 = _yield_raw_df(raw_data2, 'Equity')
data2.tail()

In [ ]:
data.columns

In [ ]:
# display the data as such as you get the sector in x-axis, the country in y-axis
# and the 1m return change as values
build_2dim_dataset(data2, x='Sector', y='Country', v='1m return', calc_type='median')

--------
## Generating the `model.py` file

Now we have all the necessary methods, you will create a file that store all these methods into a general `DataModel` class. <br/>

<img src="img/06_build_model_file.png" />